In [10]:
import pandas as pd
import polars as pl
from datetime import datetime, timedelta, date

semana = date.today().isocalendar().week

In [14]:
clubs = (
    pl.read_csv(
        "https://raw.githubusercontent.com/IvoVillanueva/acb2026/refs/heads/main/logos_calendario/clubs2026.csv"
    )
     .with_columns(
        pl.col("equipo").str.replace("Baskonia", "Baskonia Kosner")
    )
    .with_columns(
        pl.when(pl.col("equipo") == "Real Madrid")
          .then(pl.lit("#00529F"))
          .when(pl.col("equipo") == "Baskonia Kosner")
          .then(pl.lit("#000000"))
          .otherwise(pl.col("color"))
          .alias("color"),

        pl.when(pl.col("equipo") == "Baskonia Kosner")
          .then(
              pl.lit(
                  "https://static.acb.com/img/www/clubes2025/2526KosnerBaskoniaLogoNegativo.png"
              )
          )
          .otherwise(pl.col("logo"))
          .alias("logo")
    )
)

clubs

equipo,logo,color,url,abb
str,str,str,str,str
"""Barça""","""https://static.acb.com/img/www…","""#154284""","""https://www.acb.com/club/plant…","""BAR"""
"""Baskonia Kosner""","""https://static.acb.com/img/www…","""#000000""","""https://www.acb.com/club/plant…","""BKN"""
"""Bàsquet Girona""","""https://static.acb.com/img/www…","""#B32836""","""https://www.acb.com/club/plant…","""GIR"""
"""BAXI Manresa""","""https://static.acb.com/img/31/…","""#D5001E""","""https://www.acb.com/club/plant…","""BAX"""
"""Casademont Zaragoza""","""https://static.acb.com/img/www…","""#C8102E""","""https://www.acb.com/club/plant…","""CAZ"""
…,…,…,…,…
"""Río Breogán""","""https://static.acb.com/logos/1…","""#85ADEA""","""https://www.acb.com/club/plant…","""BRE"""
"""Surne Bilbao Basket""","""https://static.acb.com/img/www…","""#000000""","""https://www.acb.com/club/plant…","""SBB"""
"""UCAM Murcia""","""https://static.acb.com/img/www…","""#A51A14""","""https://www.acb.com/club/plant…","""UCM"""


In [15]:
data_eurocup = (
    pl.read_csv(
        "https://raw.githubusercontent.com/IvoVillanueva/BOXSCORES-EUROLEAGE-2025_26/refs/heads/main/data/eurocup_boxscore_2025_26.csv"
    )
    .with_columns(
        pl.col("fecha").str.to_datetime("%Y-%m-%d %H:%M:%S").alias("fecha"),
        pl.col("minutes").str.replace("DNP", "0:00").alias("minutes"),
    )
    .with_columns(
        pl.when(pl.col("team_name") == "BAXI MANRESA")
        .then(pl.lit("BAXI Manresa"))
        .otherwise(pl.col("team_name"))
        .alias("team_name"),
        # --- firstName ---
        pl.when(pl.col("player").str.contains(","))
        .then(pl.col("player").str.split(", ").list.get(1).str.to_titlecase())
        .otherwise(pl.col("player").str.to_titlecase())
        .alias("firstName"),
        # --- lastName ---
        pl.when(pl.col("player").str.contains(","))
        .then(pl.col("player").str.split(", ").list.get(0).str.to_titlecase())
        .otherwise(pl.lit(None))
        .alias("lastName"),
        # --- nombre completo ---
        pl.when(pl.col("player").str.contains(","))
        .then(
            pl.col("player").str.split(", ").list.get(1).str.to_titlecase()
            + pl.lit(" ")
            + pl.col("player").str.split(", ").list.get(0).str.to_titlecase()
        )
        .otherwise(pl.col("player").str.to_titlecase())
        .alias("player"),
        # --- minutos redondeados ---
        (
            pl.col("minutes").str.split(":").list.get(0).cast(pl.Int64)
            + pl.when(pl.col("minutes").str.split(":").list.get(1).cast(pl.Int64) >= 30)
            .then(1)
            .otherwise(0)
        ).alias("minutes"),
    )
    .filter(
        (pl.col("team_name").is_in(["BAXI Manresa"]))
        & (pl.col("ronda") == pl.col("ronda").max())
        & (pl.col("semana") == semana)
    )
    .select(
        [
            "competicion",
            "fecha",
            "semana",
            "ronda",
            pl.col("id_match").alias("partido"),
            "firstName",
            "lastName",
            "player",
            pl.col("team_name").alias("equipo"),
            pl.col("minutes").alias("min"),
            pl.col("points").alias("pts"),
            pl.col("total_rebounds").alias("reb"),
            pl.col("assistances").alias("ast"),
            pl.col("steals").alias("br"),
            pl.col("turnovers").alias("bp"),
            pl.col("blocks_favour").alias("tap"),
            pl.col("fouls_commited").alias("fa"),
            pl.col("valuation").alias("val"),
            pl.col("plusminus").alias("mm"),
        ]
    )
    .sort("player")
    .join(clubs, on="equipo")
)

data_eurocup

competicion,fecha,semana,ronda,partido,firstName,lastName,player,equipo,min,pts,reb,ast,br,bp,tap,fa,val,mm,logo,color,url,abb
str,datetime[μs],i64,i64,i64,str,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,str,str,str


In [16]:
df_partidos_eurocup = (
    data_eurocup.group_by(["player", "equipo"])
    .agg(
        [
            pl.col("min").mean(),
            pl.col("pts").mean(),
            pl.col("reb").mean(),
            pl.col("ast").mean(),
            pl.col("br").mean(),
            pl.col("bp").mean(),
            pl.col("tap").mean(),
            pl.col("fa").mean(),
            pl.col("mm").mean(),
            pl.col("val").mean(),
        ]
    )
    .sort("player")
    .join(clubs, on="equipo")
)
df_partidos_eurocup

player,equipo,min,pts,reb,ast,br,bp,tap,fa,mm,val,logo,color,url,abb
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str
